<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Business Analytics e Machine Learning Para Projetos de Data Science</font>
## <font color='blue'>Projeto 4</font>
### <font color='blue'>Data Warehousing Analytics Para Análise Geoespacial com Python e DuckDB</font>

## Instalando e Carregando os Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
%%writefile requirements.txt

duckdb
geopandas
pyarrow

Overwriting requirements.txt


In [3]:
%pip install -r requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Imports
import duckdb
import geopandas
import pyarrow

In [5]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Instalando Extensões do DuckDB

In [6]:
# Instala as extensões do DuckDB
duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql("FORCE INSTALL spatial FROM 'http://nightly-extensions.duckdb.org';")
duckdb.sql('LOAD spatial')

A extensão httpfs nos permite ler arquivos diretamente do S3, enquanto a extensão espacial, como o nome sugere, será aproveitada para executar consultas geoespaciais. Por meio desses comandos, garantimos que nossa configuração do DuckDB esteja bem equipada com as extensões necessárias para as tarefas futuras.

## Inicializando o DuckDB Para Processamento em Memória

In [7]:
# Cria uma instância de banco de dados do DuckDB com armazenamento em memória
con = duckdb.connect(database = ":memory:")

In [8]:
# Instala e carrega as extensões (uma alternativa ao que foi mostrado anteriormente)
con.install_extension('httpfs')
con.load_extension('httpfs')
con.install_extension('spatial')
con.load_extension('spatial')

## Carregando Dados Geoespaciais no Formato GeoParquet

Veja detalhes sobre a fonte de dados no Capítulo 11 do curso.

Agora que o DuckDB está configurado e pronto, a próxima etapa é carregar os dados espaciais de localização de edifícios. O conjunto de dados foi particionado usando duas estratégias diferentes:

- by country
- by country, sub-partitioned by S2 grid

O subparticionamento é usado para otimizar o desempenho na leitura de arquivos GeoParquet. Ao definir um limite de 20 milhões de áreas de construção por arquivo, mantemos o tamanho do grupo de linhas um tanto otimizado. Carregar diretamente um arquivo GeoParquet grande é muito mais lento.

Vamos primeiro carregar um país completo (grande) usando um único arquivo:

In [9]:
# Endereço do arquivo no S3 na nuvem AWS
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"

In [10]:
# Tipo de particionamento
partitions = "by_country"

In [11]:
# Código ISO do país
country_iso = "BRA"

> Criamos a tabela com os dados do Brasil.

In [12]:
%%time

con.execute(f"""
    CREATE OR REPLACE TABLE dsa_brasil_buildings AS
      SELECT * FROM parquet_scan('{prefix}/by_country_s2/country_iso={country_iso}/*.parquet')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 51.4 s, sys: 19 s, total: 1min 10s
Wall time: 2min 44s


## Consultas aos Dados Geoespaciais em Tabela de 140 Milhões de Registros

In [13]:
%%time

# Resumo da tabela criada
con.query('describe dsa_brasil_buildings')

CPU times: user 139 µs, sys: 23 µs, total: 162 µs
Wall time: 165 µs


┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ geometry       │ BLOB        │ YES     │ NULL    │ NULL    │ NULL    │
│ boundary_id    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ bf_source      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ confidence     │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ area_in_meters │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ s2_id          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ country_iso    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [14]:
%%time

# Contagem de registros
con.query('SELECT COUNT(*) FROM dsa_brasil_buildings;')

CPU times: user 54 µs, sys: 6 µs, total: 60 µs
Wall time: 62 µs


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    140368073 │
└──────────────┘

In [15]:
%%time

# Seleciona a fonte e a contagem
con.query('SELECT bf_source as data_source, COUNT(*) AS contagem FROM dsa_brasil_buildings GROUP BY data_source;')

CPU times: user 70 µs, sys: 2 µs, total: 72 µs
Wall time: 77.2 µs


┌─────────────┬───────────┐
│ data_source │ contagem  │
│   varchar   │   int64   │
├─────────────┼───────────┤
│ microsoft   │   4131217 │
│ google      │ 136236856 │
└─────────────┴───────────┘

In [16]:
%%time

# Uso do SELECT não é obrigatório com DuckDB
con.query('''FROM dsa_brasil_buildings WHERE bf_source = 'google' LIMIT 10;''')

CPU times: user 151 µs, sys: 6 µs, total: 157 µs
Wall time: 159 µs


┌─────────────────────────┬─────────────┬───────────┬────────────┬────────────────┬──────────────────────┬─────────────┐
│        geometry         │ boundary_id │ bf_source │ confidence │ area_in_meters │        s2_id         │ country_iso │
│          blob           │    int64    │  varchar  │   double   │     double     │        int64         │   varchar   │
├─────────────────────────┼─────────────┼───────────┼────────────┼────────────────┼──────────────────────┼─────────────┤
│ \x01\x03\x00\x00\x00\…  │         117 │ google    │     0.8887 │         79.488 │ -8280993814827499520 │ BRA         │
│ \x01\x03\x00\x00\x00\…  │         117 │ google    │     0.7046 │         5.8785 │ -8280993814827499520 │ BRA         │
│ \x01\x03\x00\x00\x00\…  │         117 │ google    │     0.6558 │        34.7249 │ -8280993814827499520 │ BRA         │
│ \x01\x03\x00\x00\x00\…  │         117 │ google    │     0.7123 │        23.7358 │ -8280993814827499520 │ BRA         │
│ \x01\x03\x00\x00\x00\…  │     

## Análise Estatística Por Partição dos Dados

Vamos nos aprofundar na análise das estatísticas da partição S2. Para este exercício, selecionaremos um país subdividido em múltiplas grades S2 e calcularemos a contagem de edifícios por ID da grade. Usaremos a Austrália (AUS) como nosso país de estudo devido à sua extensa distribuição geográfica e presença urbana substancial.

In [17]:
# Podemos trabalhar com dados de outros países
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country_s2"
country_iso = "AUS"

In [18]:
# Define a query SQL
dsa_query1 = f"""
    CREATE TABLE dsa_aus_buildings AS
    SELECT s2_id, COUNT(geometry) AS buildings_count
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
    GROUP BY(s2_id)
"""

In [19]:
%%time

# Executa a query
duckdb.sql(dsa_query1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 6.82 s, sys: 1.96 s, total: 8.78 s
Wall time: 16.7 s


No resultado acima, a Austrália está dividida em três redes S2 distintas, cada uma com um número variável de edifícios. Isto mostra a distribuição geográfica e a densidade dos edifícios em diferentes regiões do país.

Agora, vamos calcular o número médio de edifícios por ID de grade S2 para ter uma noção da densidade do edifício:

In [20]:
%%time

# Executa consulta
duckdb.sql("SELECT * FROM dsa_aus_buildings").show()

┌──────────────────────┬─────────────────┐
│        s2_id         │ buildings_count │
│        int64         │      int64      │
├──────────────────────┼─────────────────┤
│  3170534137668829184 │         1440431 │
│ -6052837899185946624 │          422107 │
│  7782220156096217088 │         9452987 │
└──────────────────────┴─────────────────┘

CPU times: user 640 µs, sys: 478 µs, total: 1.12 ms
Wall time: 549 µs


In [21]:
# Define a query SQL
dsa_query2 = f"""
    SELECT ROUND(AVG(buildings_count), 0) AS avg_num_buildings
    FROM dsa_aus_buildings
"""

In [22]:
%%time

# Executa a query e mostra o resultado
duckdb.sql(dsa_query2).show()

┌───────────────────┐
│ avg_num_buildings │
│      double       │
├───────────────────┤
│         3771842.0 │
└───────────────────┘

CPU times: user 468 µs, sys: 480 µs, total: 948 µs
Wall time: 464 µs


A partir dos resultados, observa-se que, em média, existem cerca de 3,7 milhões de edifícios por ID de rede. Esta métrica fornece uma estimativa aproximada da densidade de construção em diferentes segmentos geográficos na Austrália e pode servir como base para análises espaciais adicionais ou comparações com outros países.

## Executando Análise dos Dados Geoespaciais

Agora, vamos nos aprofundar em algumas análises do nosso conjunto de dados, com foco no país Lesoto (devido ao seu tamanho menor). Inicialmente, buscaremos os dados do S3 e os organizaremos em uma tabela DuckDB.

In [23]:
# Tipo de particionamento e código ISO do país
partitions = "by_country_s2"
country_iso = "LSO"

In [24]:
%%time

duckdb.sql(f"CREATE OR REPLACE TABLE dsa_lso_buildings AS SELECT * FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.58 s, sys: 199 ms, total: 4.78 s
Wall time: 5.46 s


In [25]:
%%time

# Executa a consulta
duckdb.sql(f"SELECT bf_source, COUNT(*) AS buildings_count FROM dsa_lso_buildings GROUP BY bf_source;").show()

┌───────────┬─────────────────┐
│ bf_source │ buildings_count │
│  varchar  │      int64      │
├───────────┼─────────────────┤
│ microsoft │          151722 │
│ google    │         1394189 │
└───────────┴─────────────────┘

CPU times: user 25.5 ms, sys: 1.52 ms, total: 27 ms
Wall time: 3.06 ms


Vamos comparar o conjunto de dados original do Google V3 com nosso conjunto de dados mesclado para verificar a ausência de sobreposição. Utilizaremos o conjunto de dados Open Buildings disponível na Source Cooperative para esta comparação.

Lembra que já carregamos Lesoto como uma tabela DuckDB chamada lso_buildings nas etapas anteriores? Vamos fazer o mesmo com o conjunto de dados de construção original do Google V3.

In [26]:
# Fonte de dados do Google Reasearch
prefix = "s3://us-west-2.opendata.source.coop/google-research-open-buildings/geoparquet-by-country"

In [27]:
# Código do país
country = "LS"

In [28]:
%%time

# Executa a consulta
duckdb.sql(f"CREATE OR REPLACE TABLE dsa_lso_buildings_google AS SELECT * FROM '{prefix}/country_iso={country}/{country}.parquet'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 1.07 s, sys: 335 ms, total: 1.4 s
Wall time: 2.84 s


In [29]:
%%time

# Executa a consulta
duckdb.sql("SELECT COUNT(*) FROM dsa_lso_buildings_google").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1394225 │
└──────────────┘

CPU times: user 1.17 ms, sys: 988 µs, total: 2.16 ms
Wall time: 706 µs


In [30]:
%%time

# Executa a consulta
duckdb.sql("SELECT COUNT(*) FROM dsa_lso_buildings WHERE bf_source = 'google'").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1394189 │
└──────────────┘

CPU times: user 8.76 ms, sys: 929 µs, total: 9.69 ms
Wall time: 1.18 ms


In [31]:
# Arquivo com a área de interesse (AOI)
arquivo = "limites.geojson"

In [32]:
%%time

# Executa a query
duckdb.sql(f"CREATE TABLE aoi AS SELECT * FROM ST_Read('{arquivo}')")

CPU times: user 1.05 ms, sys: 1.44 ms, total: 2.49 ms
Wall time: 1.61 ms


In [33]:
%%time

# Executa a consulta
duckdb.sql("SELECT * FROM aoi").show()

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                         geom                                                         │
│                                                       geometry                                                       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((27.492479555568025 -29.3373975037322, 27.492479555568025 -29.357824001576134, 27.523530369196408 -29.357…  │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

CPU times: user 280 µs, sys: 160 µs, total: 440 µs
Wall time: 245 µs


In [34]:
# Define a query SQL
dsa_query3 = """
CREATE TABLE dsa_lso_buildings_clipped AS
SELECT ST_Intersection(ST_GeomFromWKB(b.geometry), a.geom) AS geom, b.bf_source
FROM dsa_lso_buildings b, aoi a
WHERE ST_Intersects(ST_GeomFromWKB(b.geometry), a.geom);
"""

O código SQL acima está criando uma nova tabela chamada dsa_lso_buildings_clipped a partir de uma operação de interseção geográfica entre duas tabelas existentes: dsa_lso_buildings e aoi. Vamos compreender cada parte do código para esclarecer o que ele faz:

CREATE TABLE dsa_lso_buildings_clipped AS: Esta instrução SQL cria uma nova tabela chamada dsa_lso_buildings_clipped. O conteúdo desta tabela é o resultado da consulta que segue o AS.

SELECT:

ST_Intersection(ST_GeomFromWKB(b.geometry), a.geom) AS geom: Esta parte seleciona a interseção geométrica das colunas geometry da tabela dsa_lso_buildings e geom da tabela aoi. A função ST_Intersection calcula a área comum entre duas geometrias. ST_GeomFromWKB é uma função que converte a coluna geometry, que está em formato WKB (Well-Known Binary), para um tipo de dado geométrico que o PostGIS pode manipular.

b.bf_source: Esta coluna é selecionada diretamente da tabela dsa_lso_buildings. Ela contém informações sobre a fonte dos dados da tabela b.

FROM dsa_lso_buildings b, aoi a: Esta linha especifica as tabelas de onde os dados serão retirados. Aqui, dsa_lso_buildings é referenciada com o alias b e aoi com o alias a.

WHERE ST_Intersects(ST_GeomFromWKB(b.geometry), a.geom): A cláusula WHERE filtra os registros para incluir apenas aqueles onde as geometrias das tabelas b e a se intersectam. ST_Intersects é uma função que retorna TRUE se duas geometrias se sobrepõem de alguma forma.

Essencialmente, a consulta cria uma nova tabela contendo as áreas de sobreposição entre os edifícios da tabela dsa_lso_buildings e uma área de interesse definida na tabela aoi. Além disso, a nova tabela inclui a coluna bf_source para cada geometria resultante, indicando a origem dos dados do edifício correspondente.

In [35]:
%%time

# Executa a query
duckdb.sql(dsa_query3)

CPU times: user 799 ms, sys: 5.2 ms, total: 804 ms
Wall time: 80.8 ms


In [36]:
%%time

# Executa a consulta
duckdb.sql("SELECT COUNT(*) FROM dsa_lso_buildings_clipped").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        13561 │
└──────────────┘

CPU times: user 470 µs, sys: 325 µs, total: 795 µs
Wall time: 438 µs


In [37]:
# Define a query SQL
dsa_query4 = """
CREATE TABLE dsa_lso_buildings_google_clipped AS
SELECT ST_Intersection(ST_GeomFromWKB(b.geometry), a.geom) AS geom
FROM dsa_lso_buildings_google b, aoi a
WHERE ST_Intersects(ST_GeomFromWKB(b.geometry), a.geom);
"""

In [38]:
%%time

# Executa a query
duckdb.sql(dsa_query4)

CPU times: user 708 ms, sys: 2.05 ms, total: 710 ms
Wall time: 104 ms


In [39]:
%%time

# Executa a consulta
duckdb.sql("SELECT COUNT(*) FROM dsa_lso_buildings_google_clipped").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        13213 │
└──────────────┘

CPU times: user 420 µs, sys: 153 µs, total: 573 µs
Wall time: 333 µs


In [40]:
# Define a query SQL
dsa_query5 = """
CREATE TABLE dsa_sem_intersecao AS
SELECT m.*
FROM dsa_lso_buildings_clipped m
WHERE NOT EXISTS (
    SELECT 1
    FROM dsa_lso_buildings_google_clipped g
    WHERE ST_Intersects(m.geom, g.geom)
);
"""

A consulta dsa_query5 cria a tabela dsa_sem_intersecao contendo todas as entradas da tabela dsa_lso_buildings_clipped que não têm interseção geométrica com qualquer entrada da tabela dsa_lso_buildings_google_clipped. 

Este procedimento é útil para identificar áreas ou objetos que são exclusivos à tabela dsa_lso_buildings_clipped em relação a dsa_lso_buildings_google_clipped.

In [41]:
%%time

# Executa a query
duckdb.sql(dsa_query5)

CPU times: user 909 ms, sys: 682 µs, total: 910 ms
Wall time: 907 ms


In [42]:
%%time

# Executa a consulta
duckdb.sql("SELECT count(*) FROM dsa_sem_intersecao").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          348 │
└──────────────┘

CPU times: user 368 µs, sys: 216 µs, total: 584 µs
Wall time: 293 µs


## Exportando o Resultado da Análise Para o Formato Espacial

Concluídas nossas análises é hora de exportar os dados para os formatos de dados geoespaciais desejados. Neste caso, utilizaremos FlatGeobuf devido à sua eficiência no tratamento de dados geoespaciais!

In [43]:
%%time
output_file = "dataset1.fgb"
duckdb.sql(f"COPY (SELECT * from dsa_lso_buildings_clipped) TO '{output_file}' WITH  (FORMAT GDAL, DRIVER 'FlatGeobuf');")

CPU times: user 20.5 ms, sys: 4.54 ms, total: 25 ms
Wall time: 25.3 ms


In [44]:
%%time
output_file = "dataset2.fgb"
duckdb.sql(f"COPY (SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry from dsa_lso_buildings) TO '{output_file}' WITH  (FORMAT GDAL, DRIVER 'FlatGeobuf');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 6.21 s, sys: 858 ms, total: 7.07 s
Wall time: 5.09 s


In [45]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [46]:
#%watermark -v -m

In [47]:
#%watermark --iversions

# Fim